Find outliers in white supremacist data to probably remove  
Outliers = posts that don't represent white supremacist ideology

# Topic modeling of white supremacist data

In [1]:
# Load data
import pandas as pd

path = '../tmp/white_supremacist_train_corpus.pkl'
ws_data = pd.read_pickle(path)
ws_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 229305 entries, calderon2021_0 to stormfront_99972
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype              
---  ------      --------------   -----              
 0   text        229305 non-null  object             
 1   word_count  229305 non-null  int64              
 2   dataset     229305 non-null  object             
 3   source      229305 non-null  object             
 4   domain      229305 non-null  object             
 5   timestamp   229051 non-null  datetime64[ns, UTC]
 6   num_words   229305 non-null  int64              
dtypes: datetime64[ns, UTC](1), int64(2), object(4)
memory usage: 14.0+ MB


## BERTopic

In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mamille3/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [11]:
# Remove stopwords
import nltk
from tqdm.notebook import tqdm

stops = nltk.corpus.stopwords.words('english')
processed = [' '.join([wd for wd in x if not wd in stops]) for x in tqdm(ws_data.text.str.split())]

  0%|          | 0/229305 [00:00<?, ?it/s]

In [12]:
from bertopic import BERTopic

topic_model = BERTopic(nr_topics=20)
topics, probs = topic_model.fit_transform(processed)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [13]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,195774,-1_people_white_would_nt
1,0,3454,0_amnesty_immigration_daca_noamnesty
2,1,3152,1_germany_refugees_german_die
3,2,2247,2_nigger_niggers_nt_fucking
4,3,2047,3_said_police_murder_death
5,4,1949,4_russia_putin_ukraine_russian
6,5,1904,5_vittu_uploaded_christianity_vimeo
7,6,1854,6_israel_iran_nuclear_israeli
8,7,1821,7_illegal_ice_illegals_aliens
9,8,1455,8_women_men_white_woman


In [15]:
topic_model.save('../tmp/ws_20topics.model')

/home/mamille3/white_supremacist_lang/conda_env/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [14]:
topic_model.get_topic(-1)

[('people', 0.014556471021788791),
 ('white', 0.013345887147097588),
 ('would', 0.011943346326650358),
 ('nt', 0.011632695220991354),
 ('one', 0.01160537766890405),
 ('like', 0.011536156895394636),
 ('jews', 0.009329695022260668),
 ('us', 0.009065376588491669),
 ('even', 0.008989363302219628),
 ('get', 0.008548228021192714)]

In [ ]:
topic_model.get_topic(-1)

[('white', 0.0006610885346912826),
 ('and', 0.0006420414081201484),
 ('to', 0.0006389698439736597),
 ('that', 0.0006327855719079347),
 ('of', 0.0006302908230078038),
 ('this', 0.0006295077972314798),
 ('the', 0.0006293011789075619),
 ('it', 0.0006291463039709799),
 ('we', 0.0006286810943055862),
 ('he', 0.0006258418782667148)]

In [7]:
topic_model.reduce_topics(ws_data.text, nr_topics=50)

In [8]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,175702,-1_the_to_of_and
1,0,2706,0_russia_putin_russian_ukraine
2,1,2045,1_islam_muslims_muslim_islamic
3,2,1998,2_niggers_nigger_you_nt
4,3,1947,3_women_men_to_you
5,4,1771,4_syria_iran_isis_assad
6,5,1764,5_was_said_police_her
7,6,1714,6_she_her_fat_is
8,7,1632,7_daca_amnesty_immigration_noamnesty
9,8,1463,8_trump_cruz_he_is
